# E-COMMERCE A/B TESTING
  by Irene Undiandeye

## Task statement

You've received an analytical task from an international online store. Your predecessor failed to complete it: they launched an A/B test and then quit (to start a watermelon farm in Brazil). They left only the technical specifications and the test results. 

### Technical description

- Test name: `recommender_system_test`
- Groups: А (control), B (new payment funnel)
- Launch date: 2020-12-07
- The date when they stopped taking up new users: 2020-12-21
- End date: 2021-01-01
- Audience: 15% of the new users from the EU region
- Purpose of the test: testing changes related to the introduction of an improved recommendation system
- Expected result: within 14 days of signing up, users will show better conversion into product page views (the `product_page` event), product card views (`product_card`) and purchases (`purchase`). At each of the stage of the funnel `product_page → product_card → purchase`, there will be at least a 10% increase.
- Expected number of test participants: 6000

Download the test data, see whether it was carried out correctly, and analyze the results. 

## TABLE OF CONTENT
- Import libraries and Data
- Data preprocessing
- Data exploration
- A/B testing and Result
- Recommendation and conclusion

# Step 1 : Import libraries and Data

In [55]:
import pandas as pd
import numpy as np
from datetime import datetime
from statsmodels.stats.weightstats import ztest as ztest
import statsmodels.api as sm

In [2]:
marketing_event = pd.read_csv('ab_project_marketing_events_us.csv')
users = pd.read_csv('final_ab_new_users_upd_us.csv')
user_events = pd.read_csv('final_ab_events_upd_us.csv')
participants = pd.read_csv('final_ab_participants_upd_us.csv')

# step 2: Data preprocessing

In [3]:
marketing_event

,name,regions,start_dt,finish_dt
0,Christmas&New Year Promo,"EU, N.America",2020-12-25,2021-01-03
1,St. Valentine's Day Giveaway,"EU, CIS, APAC, N.America",2020-02-14,2020-02-16
2,St. Patric's Day Promo,"EU, N.America",2020-03-17,2020-03-19
3,Easter Promo,"EU, CIS, APAC, N.America",2020-04-12,2020-04-19
4,4th of July Promo,N.America,2020-07-04,2020-07-11
5,Black Friday Ads Campaign,"EU, CIS, APAC, N.America",2020-11-26,2020-12-01
6,Chinese New Year Promo,APAC,2020-01-25,2020-02-07
7,Labor day (May 1st) Ads Campaign,"EU, CIS, APAC",2020-05-01,2020-05-03
8,International Women's Day Promo,"EU, CIS, APAC",2020-03-08,2020-03-10
9,Victory Day CIS (May 9th) Event,CIS,2020-05-09,2020-05-11


In [4]:
marketing_event.isna().sum()

name         0
regions      0
start_dt     0
finish_dt    0
dtype: int64

In [5]:
marketing_event.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   name       14 non-null     object
 1   regions    14 non-null     object
 2   start_dt   14 non-null     object
 3   finish_dt  14 non-null     object
dtypes: object(4)
memory usage: 576.0+ bytes


In [6]:
marketing_event['name'].duplicated().sum()

0

In [7]:
marketing_event['start_dt'] = pd.to_datetime(marketing_event['start_dt'])
marketing_event['finish_dt'] = pd.to_datetime(marketing_event['finish_dt'])

In [8]:
marketing_event.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   name       14 non-null     object        
 1   regions    14 non-null     object        
 2   start_dt   14 non-null     datetime64[ns]
 3   finish_dt  14 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 576.0+ bytes


In [9]:
users

,user_id,first_date,region,device
0,D72A72121175D8BE,2020-12-07,EU,PC
1,F1C668619DFE6E65,2020-12-07,N.America,Android
2,2E1BF1D4C37EA01F,2020-12-07,EU,PC
3,50734A22C0C63768,2020-12-07,EU,iPhone
4,E1BDDCE0DAFA2679,2020-12-07,N.America,iPhone
...,...,...,...,...
58698,1DB53B933257165D,2020-12-20,EU,Android
58699,538643EB4527ED03,2020-12-20,EU,Mac
58700,7ADEE837D5D8CBBD,2020-12-20,EU,PC
58701,1C7D23927835213F,2020-12-20,EU,iPhone


In [10]:
users['first_date'] = pd.to_datetime(users['first_date'])

In [11]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58703 entries, 0 to 58702
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   user_id     58703 non-null  object        
 1   first_date  58703 non-null  datetime64[ns]
 2   region      58703 non-null  object        
 3   device      58703 non-null  object        
dtypes: datetime64[ns](1), object(3)
memory usage: 1.8+ MB


In [12]:
users.isna().sum()

user_id       0
first_date    0
region        0
device        0
dtype: int64

In [13]:
users['user_id'].duplicated().sum()

0

In [14]:
user_events

,user_id,event_dt,event_name,details
0,E1BDDCE0DAFA2679,2020-12-07 20:22:03,purchase,99.99
1,7B6452F081F49504,2020-12-07 09:22:53,purchase,9.99
2,9CD9F34546DF254C,2020-12-07 12:59:29,purchase,4.99
3,96F27A054B191457,2020-12-07 04:02:40,purchase,4.99
4,1FD7660FDF94CA1F,2020-12-07 10:15:09,purchase,4.99
...,...,...,...,...
423756,245E85F65C358E08,2020-12-30 19:35:55,login,NaN
423757,9385A108F5A0A7A7,2020-12-30 10:54:15,login,NaN
423758,DB650B7559AC6EAC,2020-12-30 10:59:09,login,NaN
423759,F80C9BDDEA02E53C,2020-12-30 09:53:39,login,NaN


In [15]:
user_events['event_dt'] = pd.to_datetime(user_events['event_dt'])

In [16]:
user_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 423761 entries, 0 to 423760
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   user_id     423761 non-null  object        
 1   event_dt    423761 non-null  datetime64[ns]
 2   event_name  423761 non-null  object        
 3   details     60314 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 12.9+ MB


In [17]:
user_events.isna().sum()

user_id            0
event_dt           0
event_name         0
details       363447
dtype: int64

In [18]:
user_events['user_id'].duplicated().sum()

365058

In [19]:

user_events.drop_duplicates(subset=['user_id'], inplace=True)

In [20]:
user_events['user_id'].duplicated().sum()

0

In [21]:
user_events.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58703 entries, 0 to 403229
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   user_id     58703 non-null  object        
 1   event_dt    58703 non-null  datetime64[ns]
 2   event_name  58703 non-null  object        
 3   details     19568 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 2.2+ MB


In [22]:
participants

,user_id,group,ab_test
0,D1ABA3E2887B6A73,A,recommender_system_test
1,A7A3664BD6242119,A,recommender_system_test
2,DABC14FDDFADD29E,A,recommender_system_test
3,04988C5DF189632E,A,recommender_system_test
4,4FF2998A348C484F,A,recommender_system_test
...,...,...,...
14520,1D302F8688B91781,B,interface_eu_test
14521,3DE51B726983B657,A,interface_eu_test
14522,F501F79D332BE86C,A,interface_eu_test
14523,63FBE257B05F2245,A,interface_eu_test


In [23]:
participants.isna().sum()

user_id    0
group      0
ab_test    0
dtype: int64

In [24]:
participants.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14525 entries, 0 to 14524
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  14525 non-null  object
 1   group    14525 non-null  object
 2   ab_test  14525 non-null  object
dtypes: object(3)
memory usage: 340.6+ KB


In [25]:
participants['user_id'].duplicated().sum()

887

In [26]:
participants.drop_duplicates(subset=['user_id'], inplace=True)

In [27]:
participants['user_id'].duplicated().sum()

0

In [28]:
participants.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13638 entries, 0 to 14524
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  13638 non-null  object
 1   group    13638 non-null  object
 2   ab_test  13638 non-null  object
dtypes: object(3)
memory usage: 426.2+ KB


In [29]:
participants.isna().sum()

user_id    0
group      0
ab_test    0
dtype: int64

In [30]:
participants

,user_id,group,ab_test
0,D1ABA3E2887B6A73,A,recommender_system_test
1,A7A3664BD6242119,A,recommender_system_test
2,DABC14FDDFADD29E,A,recommender_system_test
3,04988C5DF189632E,A,recommender_system_test
4,4FF2998A348C484F,A,recommender_system_test
...,...,...,...
14520,1D302F8688B91781,B,interface_eu_test
14521,3DE51B726983B657,A,interface_eu_test
14522,F501F79D332BE86C,A,interface_eu_test
14523,63FBE257B05F2245,A,interface_eu_test


# step 3: Data exploration

1. Study conversion at different funnel stages

In [31]:
con_stages = user_events.groupby('event_name').user_id.count().reset_index()

In [32]:
con_stages

,event_name,user_id
0,login,9092
1,product_cart,12885
2,product_page,17158
3,purchase,19568


Is the number of events per user distributed equally in the samples?

In [33]:
event_per_user = user_events.groupby('user_id').event_name.count().reset_index()

In [34]:
event_per_user

,user_id,event_name
0,0001710F4DDB1D1B,1
1,000199F1887AE5E6,1
2,0002499E372175C7,1
3,0002CE61FF2C4011,1
4,000456437D0EFFE1,1
...,...,...
58698,FFF7A7BAF44FE9B4,1
58699,FFF8FDBE2FE99C91,1
58700,FFF91B6C5431F375,1
58701,FFFAB034A09DCC28,1


How is the number of events distributed by days?

In [35]:
#extract date from the event date column
user_events["Date"] = [d.date() for d in user_events["event_dt"]]

In [36]:
user_events.head()

,user_id,event_dt,event_name,details,Date
0,E1BDDCE0DAFA2679,2020-12-07 20:22:03,purchase,99.99,2020-12-07
1,7B6452F081F49504,2020-12-07 09:22:53,purchase,9.99,2020-12-07
2,9CD9F34546DF254C,2020-12-07 12:59:29,purchase,4.99,2020-12-07
3,96F27A054B191457,2020-12-07 04:02:40,purchase,4.99,2020-12-07
4,1FD7660FDF94CA1F,2020-12-07 10:15:09,purchase,4.99,2020-12-07


In [37]:
#group by date to find the number of event distributed by date
event_dt = user_events.groupby('Date').event_name.count().reset_index()

In [38]:
event_dt

,Date,event_name
0,2020-12-07,4943
1,2020-12-08,3128
2,2020-12-09,2091
3,2020-12-10,2844
4,2020-12-11,2254
5,2020-12-12,3610
6,2020-12-13,4187
7,2020-12-14,5460
8,2020-12-15,2930
9,2020-12-16,2094


Are there users who enter both samples

# Step 4: A/B testing and Result


A/B TESTING
PURPOSE OF THE TEST: This test is aimed at finding the changes related to an introduction of a new recommender system.

- Ho: the old recommender system has more conversion at different funnel stages
- H1: the old recommender system has lesser conversion at different funnel stages

In [39]:
#make a copy of the participant dataframe and store in parti_needed
parti_needed = participants.copy()

In [40]:
parti_needed.head()

,user_id,group,ab_test
0,D1ABA3E2887B6A73,A,recommender_system_test
1,A7A3664BD6242119,A,recommender_system_test
2,DABC14FDDFADD29E,A,recommender_system_test
3,04988C5DF189632E,A,recommender_system_test
4,4FF2998A348C484F,A,recommender_system_test


In [41]:
#drop rows where ab_test is interface_eu_test
parti_needed = parti_needed.drop(parti_needed[parti_needed.ab_test == 'interface_eu_test'].index)

In [42]:
parti_needed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3675 entries, 0 to 3674
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  3675 non-null   object
 1   group    3675 non-null   object
 2   ab_test  3675 non-null   object
dtypes: object(3)
memory usage: 114.8+ KB


In [43]:
ab_fulldata =pd.merge(user_events, parti_needed, on='user_id')

In [44]:
ab_fulldata

,user_id,event_dt,event_name,details,Date,group,ab_test
0,831887FE7F2D6CBA,2020-12-07 06:50:29,purchase,4.99,2020-12-07,A,recommender_system_test
1,3C5DD0288AC4FE23,2020-12-07 19:42:40,purchase,4.99,2020-12-07,A,recommender_system_test
2,49EA242586C87836,2020-12-07 06:31:24,purchase,99.99,2020-12-07,B,recommender_system_test
3,2B06EB547B7AAD08,2020-12-07 21:36:38,purchase,4.99,2020-12-07,A,recommender_system_test
4,A640F31CAC7823A6,2020-12-07 18:48:26,purchase,4.99,2020-12-07,B,recommender_system_test
...,...,...,...,...,...,...,...
3670,928364C4C9F13FA8,2020-12-21 17:17:34,login,NaN,2020-12-21,B,recommender_system_test
3671,1484BBF124DB1B18,2020-12-21 17:51:44,login,NaN,2020-12-21,A,recommender_system_test
3672,BEF16764A13AEC34,2020-12-21 03:49:49,login,NaN,2020-12-21,B,recommender_system_test
3673,1E6B9298415AA97A,2020-12-21 18:20:12,login,NaN,2020-12-21,B,recommender_system_test


In [45]:
ab_masterdata =pd.merge(ab_fulldata, users, on='user_id')

In [46]:
ab_masterdata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3675 entries, 0 to 3674
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   user_id     3675 non-null   object        
 1   event_dt    3675 non-null   datetime64[ns]
 2   event_name  3675 non-null   object        
 3   details     1128 non-null   float64       
 4   Date        3675 non-null   object        
 5   group       3675 non-null   object        
 6   ab_test     3675 non-null   object        
 7   first_date  3675 non-null   datetime64[ns]
 8   region      3675 non-null   object        
 9   device      3675 non-null   object        
dtypes: datetime64[ns](2), float64(1), object(7)
memory usage: 315.8+ KB


In [47]:
ab_masterdata['group'].value_counts()

A    2747
B     928
Name: group, dtype: int64

In [48]:
groups_pivot_tbl= ab_masterdata.pivot_table(index="event_name", values="user_id", columns= "group", aggfunc="nunique").reset_index()
groups_pivot_tbl

group,event_name,A,B
0,login,512,228
1,product_cart,557,177
2,product_page,806,267
3,purchase,872,256


In [49]:
#The number of participant shown the old recomender system
control_group = (ab_masterdata['group'] == 'A').sum()
control_group

2747

In [50]:
#The number of participant shown the new reccomender system
Treament_group =(ab_masterdata['group'] == 'B').sum()
Treament_group

928

Caculating z test for log in stage converstion

In [51]:
#z test for log in converstion
longinA=len(ab_masterdata[ (ab_masterdata.group == 'A')  &  (ab_masterdata.event_name == 'login')])
print("the number of conversions with the group A for login event is   ", longinA)
longinB=len(ab_masterdata[ (ab_masterdata.group == 'B')  &  (ab_masterdata.event_name == 'login')])
print("the number of conversions with the group B for login event is   ", longinB)



Ztestlogin = sm.stats.proportions_ztest([longinB, longinA], [Treament_group, control_group], alternative='larger')
print("the zscore and P-value for the login turnel is   ", Ztestlogin )

the number of conversions with the group A for login event is    512
the number of conversions with the group B for login event is    228
the zscore and P-value for the login turnel is    (3.894931124061927, 4.911331859406934e-05)


Caculating z test for product cart converstion

In [52]:
#z test for product_cart converstion
product_cartA=len(ab_masterdata[ (ab_masterdata.group == 'A')  &  (ab_masterdata.event_name == 'product_cart')])
print("the number of conversions with the group A for product_cart event is   ", product_cartA)
product_cartB=len(ab_masterdata[ (ab_masterdata.group == 'B')  &  (ab_masterdata.event_name == 'product_cart')])
print("the number of conversions with the group B for product_cart event is   ", product_cartB)

Ztestproduct_cart = sm.stats.proportions_ztest([product_cartB, product_cartA], [ Treament_group, control_group], alternative='larger')
print("the zscore and P-value for the product_cart turnel is   ", Ztestproduct_cart )


the number of conversions with the group A for product_cart event is    557
the number of conversions with the group B for product_cart event is    177
the zscore and P-value for the product_cart turnel is    (-0.7927617688494452, 0.7860416826426335)


Caculating z test for product page converstion

In [58]:
product_pageA=len(ab_masterdata[ (ab_masterdata.group == 'A')  &  (ab_masterdata.event_name == 'product_page')])
print("the number of conversions with the group A for product_page event is   ", product_pageA)
product_pageB=len(ab_masterdata[ (ab_masterdata.group == 'B')  &  (ab_masterdata.event_name == 'product_page')])
print("the number of conversions with the group B for product_page event is   ", product_pageB)

Ztestproduct_page = z_score, p_value = sm.stats.proportions_ztest([product_pageB, product_pageA], [Treament_group, control_group], alternative='larger')
print("the zscore and P-value for the product_page is   ", Ztestproduct_page )

the number of conversions with the group A for product_page event is    806
the number of conversions with the group B for product_page event is    267
the zscore and P-value for the product_page is    (-0.32991993044244, 0.6292697682080151)


In [57]:
purchaseA=len(ab_masterdata[ (ab_masterdata.group == 'A')  &  (ab_masterdata.event_name == 'purchase')])
print("the number of conversions with the group A for purchase event is   ", purchaseA)
purchaseB=len(ab_masterdata[ (ab_masterdata.group == 'B')  &  (ab_masterdata.event_name == 'purchase')])
print("the number of conversions with the group B for product_cart event is   ", purchaseB)

Ztestpurchase = sm.stats.proportions_ztest([purchaseB, purchaseA], [Treament_group, control_group], alternative='larger')
print("the zscore and P-value for the purchase turnel is   ", Ztestpurchase )

the number of conversions with the group A for purchase event is    872
the number of conversions with the group B for product_cart event is    256
the zscore and P-value for the purchase turnel is    (-2.3740870442615747, 0.9912037986683426)


HYPOTHESIS:
- H0: The number of conversions of group B (Treatment group) is equal or less than that of group A (control group)
- H1: The number of conversions of group B (Treatment group) is greater than that of group A (control group)

DECISION RULE:
- Accept null hypothesis if p-vaue is greater than alpha
- Reject null hypothesis if p-value is less than alpha

WHERE:
- alpha = 0.05


In [ ]:
print("the zscore and P-value for the login turnel is   ", Ztestlogin )
print("the zscore and P-value for the product_cart turnel is   ", Ztestproduct_cart )
print("the zscore and P-value for the product_page is   ", Ztestproduct_page )
print("the zscore and P-value for the purchase turnel is   ", Ztestpurchase )

In [ ]:
ab_masterdata.head()

In [ ]:
print('Min : {}, Max : {}'.format(min(ab_masterdata.Date), max(ab_masterdata.Date)))

# Step 5: Recommendation and conclusion


The results of the T-test show that the p-value of all the stages of the funnel is greater than alpha (0.05) hence we accept the null hypothesis implying That the number of conversions of group B (Treatment group) is equal to or less than that of group A (control group). therefore, then we can say the old recommender system is better than the new one.

CONCLUSION

Though the analysis shows that the old recommender system is better, the general technical description and dataset have the following limitations
- Expected number of test participants: 6000 but in the dataset, the number of the participant of the "recommender_system_test" are 3675. this could mean that the test was not completed.
- Launch date: 2020-12-07 End Date: 2021-01-01
the dataset has a lunch date of 2020-12-07 as stated in the technical description but the end date for data collection was stated to be 2021-01-01 the test was not carried out up to the planned end date. that explains why the number of participants is not up to 6000.
- the difference in the number of people shown the new recommender system(2747) and the number of participants shown the new recommender system(928) is too wide.

RECOMMENDATIONS
- because of this limitation mentioned above, I consider the test not accurate in deciding which of the recommender system is better and so I recommend that the company should begin a new test following all the necessary technical description.
